In [1]:
NPROCS=14

hamiltonian_encodings= [(0, 1, 2, 4, 6, 0, 1, 3, 5, 7), (0, 1, 2, 4, 6, 1, 2, 3, 5, 7), (0, 1, 2, 4, 6, 1, 3, 4, 5, 7), (0, 2, 3, 4, 6, 1, 2, 3, 5, 7), (0, 2, 3, 4, 6, 1, 3, 4, 5, 7), (0, 2, 4, 5, 6, 1, 3, 4, 5, 7), (0, 1, 2, 4, 6, 0, 2, 3, 5, 7), (0, 1, 2, 4, 6, 0, 3, 4, 5, 7), (0, 1, 2, 4, 6, 2, 3, 4, 5, 7), (0, 2, 3, 4, 6, 0, 2, 3, 5, 7), (0, 2, 4, 5, 6, 0, 2, 3, 5, 7), (0, 2, 3, 4, 6, 0, 3, 4, 5, 7), (0, 2, 4, 5, 6,  0, 3, 4, 5, 7), (0, 2, 3, 4, 6, 2, 3, 4, 5, 7), (0, 2, 4, 5, 6, 2, 3, 4, 5, 7), (1, 2, 3, 4, 6, 0, 2, 3, 5, 7), (1, 2, 4, 5, 6, 0, 3, 4, 5, 7), (1, 2, 3, 4, 6, 0, 3, 4, 5, 7), (1, 2, 3, 4, 6, 2, 3, 4, 5, 7), (1, 2, 4, 5, 6, 2, 3, 4, 5, 7), (2, 3, 4, 5, 6, 2, 3, 4, 5, 7)]

from gcaops.graph.formality_graph import FormalityGraph
def encoding_to_graph(encoding):
    targets = [encoding[0:5], encoding[5:10]]
    edges = sum([[(k+1,v) for v in t] for (k,t) in enumerate(targets)], [])
    return FormalityGraph(0, 8, edges)

hamiltonian_graphs = [encoding_to_graph(e) for e in hamiltonian_encodings]
print("Number of 5D Hamiltonians:", len(hamiltonian_graphs), flush=True)

from gcaops.algebra.differential_polynomial_ring import DifferentialPolynomialRing
from gcaops.algebra.superfunction_algebra import SuperfunctionAlgebra

D5 = DifferentialPolynomialRing(QQ, ('rho','a1','a2', 'a3'), ('x','y','z','w', 'v'), max_differential_orders=[3+1,1+3+1,1+3+1, 1+3+1])
rho, a1, a2, a3 = D5.fibre_variables()
x,y,z,w, v = D5.base_variables()
even_coords = [x,y,z,w,v]

S5.<xi0,xi1,xi2,xi3,xi4> = SuperfunctionAlgebra(D5, D5.base_variables())
xi = S5.gens()
odd_coords = xi
epsilon = xi[0]*xi[1]*xi[2]*xi[3]*xi[4] 

import itertools
from multiprocessing import Pool

def evaluate_graph(g):
    E = x*xi[0] + y*xi[1] + z*xi[2] + w*xi[3] + v*xi[4] 
    result = S5.zero()
    for index_choice in itertools.product(itertools.permutations(range(5)), repeat=2):
        sign = epsilon[index_choice[0]] * epsilon[index_choice[1]]
        vertex_content = [S5(rho), S5(rho), S5(a1), S5(a1), S5(a2), S5(a2), S5(a3), S5(a3)]
        for ((source, target), index) in zip(g.edges(), sum(map(list, index_choice), [])):
            vertex_content[target] = vertex_content[target].derivative(even_coords[index])
        result += sign * prod(vertex_content)
    return result

print("Calculating 5D hamiltonian_formulas", flush=True)
hamiltonian_formulas = []
with Pool(processes=NPROCS) as pool:
    hamiltonian_formulas = list(pool.imap(evaluate_graph, hamiltonian_graphs))
print("Calculated 5D hamiltonian_formulas", flush=True)

hamiltonian_formulas.count(0)
[k for (k, ham) in enumerate(hamiltonian_formulas) if ham == 0]

from itertools import combinations
for i,j in combinations(range(len(hamiltonian_formulas)),2):
    if i!=j and hamiltonian_formulas[i]==hamiltonian_formulas[j]:
        print("5D Hamiltonians", i,j, "share the same formula")

for i,j in combinations(range(len(hamiltonian_formulas)),2):
    if i!=j and hamiltonian_formulas[i]==-hamiltonian_formulas[j]:
        print("5D Hamiltonians", i,j, "share the same formula with a - sign")

Number of 5D Hamiltonians: 21
Calculating 5D hamiltonian_formulas
Calculated 5D hamiltonian_formulas
5D Hamiltonians 3 15 share the same formula
5D Hamiltonians 4 17 share the same formula
5D Hamiltonians 5 16 share the same formula
5D Hamiltonians 10 11 share the same formula
5D Hamiltonians 13 18 share the same formula
5D Hamiltonians 14 19 share the same formula
5D Hamiltonians 1 6 share the same formula with a - sign
5D Hamiltonians 2 7 share the same formula with a - sign
